In [1]:
%matplotlib inline

from GeoToolkit.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import matplotlib.pyplot as plt
from GeoToolkit.graphics import graphics
import numpy as np
import scipy.sparse as sp
from scipy.spatial import Delaunay
from skimage import exposure
from geoh5io.workspace import Workspace
from geoh5io.objects import Surface

In [2]:
download = DataIO.fetchData(path=r"C:\Users\DominiqueFournier\Documents\GIT\GeoToolkit\Notebooks\Output", localCloud='Local')
display(download)

interactive(children=(RadioButtons(description='File Type:', options=('Local', 'Cloud'), value='Local'), Text(…

In [3]:
if download.result is not None:
    if (download.result[1] == "CSV")  :
        grid = Simulator.dataGriddingWidget(download.result[0], marker=False, dataColumn=3, Resolution=100, Method='linear')
        display(grid)
    else:
        grid = Simulator.dataGridGeoref(download.result[0])
        display(grid)

interactive(children=(Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'Spectr…

In [5]:
if download.result is not None:
    selection = Simulator.setDataExtentWidget(grid.result)
    display(selection)

interactive(children=(FloatSlider(value=-5.389099242057903e-15, continuous_update=False, description='East', m…

In [29]:

sub = 5

grid_shape = selection.result.values.shape
values = (selection.result.values.copy())
indNan = np.isnan(values)
values[indNan] = 0
values = values[::sub, ::sub].ravel()


theta = selection.result.gridCC[:, 0].reshape(grid_shape, order='F')[::sub, ::sub].ravel()
phi = selection.result.gridCC[:, 1].reshape(grid_shape, order='F')[::sub, ::sub].ravel()


R = 5380
R = (R+(values)/10)
x = R * np.cos(np.deg2rad(phi)) * np.cos(np.deg2rad(theta))
y = R * np.cos(np.deg2rad(phi)) * np.sin(np.deg2rad(theta))
z = R * np.sin(np.deg2rad(phi))

# lon = np.deg2rad(27)#np.deg2rad(15)
# lat = np.deg2rad(67.7)#np.deg2rad(30)
# r = 6000
# xloc = r * np.cos(lat) * np.cos(lon)
# yloc = r * np.cos(lat) * np.sin(lon)
# zloc = r * np.sin(lat)

# Triangulate the surface
surf = Delaunay(np.c_[theta, phi])

In [30]:
# Interpolate values on cell
values = np.mean(np.c_[
                     values[surf.simplices[:, 0]],
                     values[surf.simplices[:, 1]],
                     values[surf.simplices[:, 2]]
                 ], axis=1
)



In [32]:
# Create a geoh5 surface and export
workspace = Workspace("EMAG_V2.geoh5")
surface, data = Surface.create(
    workspace, vertices=np.c_[x, y, z], 
    cells=surf.simplices,
    
     data={"TMI": ["CELL", values]})
workspace.save_entity(surface)
workspace.finalize()


# Repeat for EWGM2012 (GRAV)

In [8]:
# from matplotlib import cm
# from skimage import exposure

# values = (selection.result.values.copy())
# indNan = np.isnan(values)
# values[indNan] = 0

# vmin, vmax = -200, 500

# cdf, bins = exposure.cumulative_distribution(
#         (values[(values < vmax) *
#                        (values > vmin)]), nbins=256
# )
# cmap = graphics.equalizeColormap('Spectral_r', bins, cdf)
# # rgb = cmap.to_rgba((((values.T)).flatten(order='F')))
# # # cmap.set_clim(vmin=0, vmax=1000)


# # rgb[indNan.reshape(-1),:] = np.nan

# # texture = pv.numpy_to_texture(rgb[:,:3].reshape((values.shape[0], values.shape[1], 3), order='F'))


# theta = (selection.result.gridCC[:, 0].reshape(values.shape, order='F'))
# phi = (selection.result.gridCC[:, 1].reshape(values.shape, order='F'))


# R = 5380
# R = (R+(values)/5)
# x = R * np.cos(np.deg2rad(phi)) * np.cos(np.deg2rad(theta))
# y = R * np.cos(np.deg2rad(phi)) * np.sin(np.deg2rad(theta))
# z = R * np.sin(np.deg2rad(phi))

# # x = selection.result.gridCC[:, 0].reshape(values.shape, order='F')
# # y = selection.result.gridCC[:, 1].reshape(values.shape, order='F')
# # z = values/5

# x[indNan] = np.nan
# values[indNan] = np.nan
# pvgrid = pv.StructuredGrid(
#     x.T, 
#     y.T, 
#     z.T)
# # grid.plot()
# # grid.warp_by_scalar()
# plotter = pv.Plotter(notebook=False)
# # pvgrid.texture_map_to_plane(inplace=True)
# plotter.add_mesh(pvgrid, scalars=values, cmap=cmap)

# # plotter.export_vtkjs("EMAGv2_NorthAtlantic")
# # plotter.show(auto_close=False)
# # plotter.export_vtkjs("EMAGv2_NorthAtlantic")
# # plotter.close()
# # plotter.show(
# #     cpos=[(5129.611755301267, 2157.873161782082, 5926.635574173455),
# #  (2089.574112805277, -37.952885761104625, 4555.515700679695),
# #  (-0.2767863524236539, -0.20325006175741522, 0.9391904639143307)], 
# #     screenshot="EMAG90d.png")
# # plotter.screenshot("EMAG90d.png")
# lon = np.deg2rad(27)#np.deg2rad(15)
# lat = np.deg2rad(67.7)#np.deg2rad(30)
# r = 6000
# xloc = r * np.cos(lat) * np.cos(lon)
# yloc = r * np.cos(lat) * np.sin(lon)
# zloc = r * np.sin(lat)

# plotter.show(cpos=[(xloc, yloc, zloc),
#  (0, 0, 500),
#  (-0., 0., 1)],screenshot="EWGM2012_15d_Europe.png")
# # Large view
# # plotter.show(cpos=[(6053.18060192332, 1419.8421698073148, 6415.325176210674),
# #  (2044.2806608584788, -10.673701877581124, 4612.252867030859),
# #  (-0.3596642657094726, -0.15237282916552064, 0.9205564278759674)])

# # Middle view
# # plotter.show(cpos=[(3187.0998318049637, 2081.707503888414, 5001.011871672062),
# #  (1848.549824736279, 751.5975240218856, 4727.89196482181),
# #  (-0.05526397612824547, -0.14716860756088546, 0.9875663491082951)])


# # Small view
# # Read the GeoTIFF as a vtkTexture:


# # plotter.show(cpos=[(2481.6857948002835, 1275.3562368857397, 5078.085082348304),
# #  (1712.322032440504, 864.7372440254018, 4902.9769571772285),
# #  (-0.1761321093785013, -0.08808508776671177, 0.9804175117566039)],screenshot="EWGM2012_Kevitsa.png")


# # Cartesian Kevitsa plotter
# # plotter.show( cpos=[(3503537.9917390947, 7495043.372086444, 11444.588074577092),
# #  (3498073.9770745155, 7510717.605823356, 435.0068286203108),
# #  (-0.06888187753633372, 0.5572240290585513, 0.8275002528016681)]
# #             )

[(1231.4457588686, -454.90740190488145, 9441.284918312835),
 (-175.24263603459463, -1318.4813079291368, 327.81394328350405),
 (-0.8833888161359897, -0.43374442628583354, 0.17745414109517918)]